In [ ]:
from ctraptools.kymos.detect import Detector
from lumicks import pylake

import ctraptools.kymos.io as kio
# import ctraptools.kymos.stepfinding as ksf
import os

In [ ]:
# Parameters
path = ""
channel = 0

In [ ]:
# Function to process single image
def process_image(image,channel,output_name):
    # Initialising Detector
    detector = Detector(half_t_w=3,peak_det_thresh = 0.06, n_max=10, c_ub=5, c_def=3, min_track_length = 10, track_heritage_weight=100)

    # Detect peaks and track
    tracks = detector.detect(image)

    # Measuring intensity
    for track in tracks.values():
        track.measure_intensity(image)
    
    # # Step fitting (to be added later)
    # ksf.fit_steps(tracks,image)

    # Saving files
    kio.save_overlay(tracks,image,output_name+"_overlay")
    kio.write_traces(tracks,output_name+"_traces")
    kio.save_plots(tracks,output_name+"_traces")
    kio.write_change_points(tracks,output_name+'_steps')

In [ ]:
# Loading image and calculating paths
for f in os.listdir(path):
    name = os.path.basename(os.path.splitext(f)[0])
    if os.path.splitext(f)[1] == ".tif":
        image = kio.read_image(path+f,channel,x_range=None)
        output_name = path+name+'_C'+str(channel)
        process_image(image, channel, output_name)
    elif os.path.splitext(f)[1] == ".h5" or os.path.splitext(f)[1] == ".hdf5":
        file = pylake.File(path+f)
        for (idx,kymo) in enumerate(file.kymos.values()):
            image = kymo.get_image()[:,:,channel]
            output_name = path+name+'_C'+str(channel)+'_kymo'+str(idx+1)
            process_image(image, channel, output_name)        
    else:
        continue